In [12]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, applications, Input
import matplotlib.pyplot as plt
import numpy as np


# Caricamento del dataset
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()


# Normalizzazione delle immagini (da valori 0-255 a valori tra 0 e 1)
train_images = train_images.astype("float32") / 255
test_images = test_images.astype("float32") / 255

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
print("Dataset:")
print(f"- Training set: {train_images.shape[0]}") #insieme di dati usato per addestrare il modello di machine learning
print(f"- Test set: {train_images.shape[0]}") #insieme di dati usato per valutare le prestazioni di un modello di machine learning
print(f"- Dimensione immagini: {train_images.shape[1:]}") #stampa le dimensioni delle immagini nel training set

print("CLASSI:")
classi = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

#stampo le immagini
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i])
    plt.xlabel(classi[train_labels[i][0]])
plt.show()

In [ ]:
# Ridefinizione delle etichette per avere un array monodimensionale
train_labels = train_labels.flatten()
test_labels = test_labels.flatten()

# Definizione della rete feed-forward
input_layer = layers.Input(shape=(32, 32, 3))
x = layers.Flatten()(input_layer)  # Flatten delle immagini
x = layers.Dense(256, activation='relu')(x)  # Primo strato denso con più neuroni
x = layers.Dense(128, activation='relu')(x)  # Secondo strato denso
x = layers.Dense(32, activation='relu')(x)   # Terzo strato più piccolo
output_layer = layers.Dense(10, activation='softmax')(x)  # Strato di output

# Creazione del modello
model = models.Model(inputs=input_layer, outputs=output_layer)

# Stampa il sommario del modello
model.summary()

In [ ]:
# Compilazione del modello con ottimizzatore e funzione di perdita modificata
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

epochs = 10
# Addestramento del modello con numero di epoche ridotto e batch modificato
print("Avvio dell'addestramento del modello...")
train_history = model.fit(train_images, train_labels, epochs=epochs, batch_size=32, validation_split=0.3)

In [ ]:
# Visualizzazione delle curve di apprendimento
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)  # in una griglia 1x2 inserisci in posizione 1
plt.plot(range(1, epochs + 1), train_history.history['loss'], label='Training Loss')
plt.plot(range(1, epochs + 1), train_history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Visualizzazione dell'accuracy
plt.subplot(1, 2, 2)  # in una griglia 1x2 inserisci in posizione 2
plt.plot(range(1, epochs + 1), train_history.history['accuracy'], label='Training Accuracy')
plt.plot(range(1, epochs + 1), train_history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Valutazione del modello su dati di test
test_loss, test_acc = model.evaluate(test_images, test_labels, batch_size=32)
print(f"Dimensione del dataset di test: {test_images.shape}")
print(f"Accuratezza sul test set: {test_acc:.4f}")

In [ ]:
#ora inizio la rete convoluzionale
# Definizione della rete convoluzionale
input_layer = layers.Input(shape=(32, 32, 3))
x = layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same')(input_layer)
x = layers.MaxPooling2D(pool_size=(2,2))(x)
x = layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D(pool_size=(2,2))(x)
x = layers.Conv2D(filters=256, kernel_size=(3,3), activation='relu', padding='same')(x)
x = layers.GlobalAveragePooling2D()(x)  # GlobalAveragePooling invece di Flatten
x = layers.Dense(256, activation='relu')(x)
output_layer = layers.Dense(10, activation='softmax')(x)

# Creazione del modello
model = models.Model(inputs=input_layer, outputs=output_layer)

# Stampa il sommario del modello
model.summary()

In [ ]:
# Compilazione del modello con ottimizzatore e funzione di perdita modificata
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),  # Learning rate ridotto
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=["accuracy"])

# Addestramento del modello con batch più piccolo e validation split aumentato
cnn_history = model.fit(train_images, train_labels,
                    epochs=15, batch_size=64, validation_split=0.3)